# Scikit-Learn IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on a kubernetes cluster
 
## Dependencies

 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install sklearn
pip install seldon-core
```

 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon-system -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon-system 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:8080`
 

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Create Seldon Core config file

In [ ]:
%%writefile sklearn_iris_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
spec:
  name: sklearn-iris-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.3
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

In [ ]:
!kubectl create -f sklearn_iris_deployment.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example \
                                 -o jsonpath='{.items[0].metadata.name}')

In [ ]:
for i in range(60):
    state = !kubectl get sdep seldon-deployment-example -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

In [ ]:
res = !curl  -s http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions -H "Content-Type: application/json" -d '{"data":{"ndarray":[[5.964,4.006,2.081,1.031]]}}'

In [ ]:
res

In [ ]:
print(res)
import json

j = json.loads(res[0])
assert j["data"]["ndarray"][0][0] > 0.0

REST request with raw_data

In [ ]:
from seldon_core.seldon_client import SeldonClient

sc = SeldonClient(deployment_name="seldon-deployment-example", namespace="seldon")
res = sc.predict(
    gateway="istio",
    gateway_endpoint="localhost:8003",
    transport="rest",
    raw_data={"data": {"ndarray": [[5.964, 4.006, 2.081, 1.031]]}},
)
print(res.response)
assert res.success == True

gRCP request with proto raw_data

In [ ]:
from seldon_core.utils import json_to_seldon_message

proto_raw_data = json_to_seldon_message(
    {"data": {"ndarray": [[5.964, 4.006, 2.081, 1.031]]}}
)
res = sc.predict(
    gateway="istio",
    gateway_endpoint="localhost:8003",
    transport="grpc",
    raw_data=proto_raw_data,
)
print(res)
assert res.success == True

In [ ]:
!kubectl delete -f sklearn_iris_deployment.yaml